# 1 Introduction to Docker: Sample App

## 1.1 Sample App

In [1]:
cd app

The sample app is a http server written in go.

In [2]:
cat main.go

package main

import (
    "fmt"
    "net/http"
)

func handler(w http.ResponseWriter, r *http.Request) {
    fmt.Fprintf(w, "Hello!")
}

func main() {
    http.HandleFunc("/", handler)
    http.ListenAndServe(":8888", nil)
}


Dockerfile looks like this,

In [3]:
cat Dockerfile

FROM golang:1.10.1-alpine as build

WORKDIR /go/src/github.com/sakajunquality/hello-go-docker
COPY main.go main.go

RUN go get -d -v ./...
RUN go install -v ./...

FROM alpine
RUN apk add --no-cache ca-certificates
COPY --from=build /go/bin/hello-go-docker /usr/local/bin/hello-go-docker

CMD ["hello-go-docker"]


## 1.2 Build the image

In [4]:
docker image build --no-cache -t k8s-hands-on:v1 .

Sending build context to Docker daemon  3.072kB
Step 1/9 : FROM golang:1.10.1-alpine as build
 ---> 41bf3e9b9f3c
Step 2/9 : WORKDIR /go/src/github.com/sakajunquality/hello-go-docker
 ---> Running in bde538b229d1
Removing intermediate container bde538b229d1
 ---> d4ab11736a1c
Step 3/9 : COPY main.go main.go
 ---> f79ad2bec090
Step 4/9 : RUN go get -d -v ./...
 ---> Running in a4908d98b14e
Removing intermediate container a4908d98b14e
 ---> 8c60232df29d
Step 5/9 : RUN go install -v ./...
 ---> Running in 90405fea99fe
github.com/sakajunquality/hello-go-docker
Removing intermediate container 90405fea99fe
 ---> 4dd649354721
Step 6/9 : FROM alpine
 ---> 3fd9065eaf02
Step 7/9 : RUN apk add --no-cache ca-certificates
 ---> Running in a0eaedeb4ea9
fetch http://dl-cdn.alpinelinux.org/alpine/v3.7/main/x86_64/APKINDEX.tar.gz
fetch http://dl-cdn.alpinelinux.org/alpine/v3.7/community/x86_64/APKINDEX.tar.gz
(1/1) Installing ca-certificates (20171114-r0)
Executing busybox-1.27.2-r7.trigger
Executing ca

## 1.3 Run as container

In [5]:
docker container run -d --rm -p 8888:8888 --name k8s-hands-on k8s-hands-on:v1

d6256c14c19f77b81fac74d6dcf12b2686fca5b471909a56b8edc0197cee4a49


Open [http://localhost:8888/](http://localhost:8888/) in your browser.

check the docker process,

In [6]:
docker ps

CONTAINER ID        IMAGE                         COMMAND                  CREATED             STATUS              PORTS                              NAMES
d6256c14c19f        k8s-hands-on:v1               "hello-go-docker"        10 seconds ago      Up 8 seconds        0.0.0.0:8888->8888/tcp             k8s-hands-on
60e18ba7b362        sakajunquality/k8s-hands-on   "dockerd-entrypoint.…"   2 hours ago         Up 2 hours          2375/tcp, 0.0.0.0:8080->8080/tcp   relaxed_brattain


## 1.4 Kill the container

In [7]:
docker kill k8s-hands-on

k8s-hands-on


check the process again,

In [8]:
docker ps

CONTAINER ID        IMAGE                         COMMAND                  CREATED             STATUS              PORTS                              NAMES
60e18ba7b362        sakajunquality/k8s-hands-on   "dockerd-entrypoint.…"   2 hours ago         Up 2 hours          2375/tcp, 0.0.0.0:8080->8080/tcp   relaxed_brattain
